In [23]:
%reload_ext autoreload
%autoreload 1
import torch 
import sys
sys.path.append('..')
from torch import nn 
from torch.nn import functional as F
from torch import optim
from utils.loader import load
# from models.vince_models import Net2B

In [24]:
# load the dataset
train_input,train_target, train_classes, test_input, test_target, test_classes = load()
train_target=train_target.float() # necessary ?
test_target=test_target.float()

In [47]:
# weight sharing with a binary classifier
# baseline model
class Net_shared(nn.Module):

    def __init__(self, nb_hidden, name = None):
        
        super(Net_shared, self).__init__()
        self.name = name
        self.conv1 = nn.Conv2d(1, 32, kernel_size = 3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 3)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(400, 1)
        #self.dropout = nn.Dropout()
        self.sigmoid = nn.Sigmoid()  
        
    def forward(self, x):
        
        x_1 = x [:, 0, :, :].view(-1, 1, 14, 14) # channel 1
        x_2 = x [:, 1, :, :].view(-1, 1, 14, 14) # channel 2
    
        x_1 = F.relu(F.max_pool2d(self.conv1(x_1), kernel_size = 3, stride = 1))
        x_1 = F.relu(F.max_pool2d(self.conv2(x_1), kernel_size = 3, stride = 3))
        x_1 = F.relu(self.fc1(x_1.view(-1, 256)))
        
        x_2 = F.relu(F.max_pool2d(self.conv1(x_2), kernel_size = 3, stride = 1))
        x_2 = F.relu(F.max_pool2d(self.conv2(x_2), kernel_size = 3, stride = 3))
        x_2 = F.relu(self.fc1(x_2.view(-1, 256)))
        
        y = torch.cat([x_1, x_2], 1)
        y = self.fc2(y)
        
        
        return self.sigmoid(y)
    

In [48]:
def train_binary(model, epoch, train_input, train_target, batch_size, 
                 criterion = nn.BCELoss(), optimizer = optim.SGD, 
                 eta = 1e-1, penalty = None, lambda_l2 = 1e-3):  
    # train a binary classifier by minimizing the binary cross entropy loss of a sigmoid output
    # not BCELossWithLogits because sigmoid activation of output already applied in model architecture
    model.train()
    optimizer = optimizer(model.parameters(), lr = eta)
    train_loss = 0
    
    for input_, target in (zip(train_input.split(batch_size), train_target.split(batch_size))):
        output = model(input_)
        batch_loss = criterion(output, target.unsqueeze(1))
        
        if penalty is not None:
            for p in model.parameters():
                batch_loss += lambda_l2 * p.pow(2).sum() # L2 penalty term
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        train_loss += batch_loss   
    print('Train Epoch: {}  | Loss {:.6f}'.format(
                epoch, train_loss.item()))
    
    

def test_binary(model, test_input, test_target, batch_size, criterion = nn.BCELoss()):
    # test a binary classifier using sigmoid followed by rounding ptobability to 0 or 1
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        print('Model Name : {}'.format(model.name))
        
        for data, target in zip(test_input.split(batch_size), test_target.split(batch_size)):
            output = model(data)
            batch_loss = criterion(output, target.unsqueeze(1))
            test_loss += batch_loss
            pred = output 
            correct += torch.round(output).squeeze().eq(target).sum() # only need to round because sigmoid already applied in model 
        print('\nTest set:  Loss: {:.4f}, Accuracy: {:.0f}%\n'.format(
        test_loss.item(), 100 * correct/len(test_target)))

In [49]:
###############################
######  Weight Sharing   ######
###############################

# initialize models and normalize data
mean, std = train_input.mean(), train_input.std()
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

model = Net_shared(100, 'net2c')



# train models
nb_epochs = 50

print('Model Name : {}'.format(model.name))

for epoch in range(1, nb_epochs):
    train_binary(model, epoch, train_input, train_target, 100)



    

Model Name : net2c
Train Epoch: 1  | Loss 6.899056
Train Epoch: 2  | Loss 6.673351
Train Epoch: 3  | Loss 6.380344
Train Epoch: 4  | Loss 5.862628
Train Epoch: 5  | Loss 5.171149
Train Epoch: 6  | Loss 4.585419
Train Epoch: 7  | Loss 4.159502
Train Epoch: 8  | Loss 3.828558
Train Epoch: 9  | Loss 3.574590
Train Epoch: 10  | Loss 3.366674
Train Epoch: 11  | Loss 3.188885
Train Epoch: 12  | Loss 3.023263
Train Epoch: 13  | Loss 2.868774
Train Epoch: 14  | Loss 2.735623
Train Epoch: 15  | Loss 2.599420
Train Epoch: 16  | Loss 2.470345
Train Epoch: 17  | Loss 2.337206
Train Epoch: 18  | Loss 2.213586
Train Epoch: 19  | Loss 2.122209
Train Epoch: 20  | Loss 2.004921
Train Epoch: 21  | Loss 1.858419
Train Epoch: 22  | Loss 1.754102
Train Epoch: 23  | Loss 1.686171
Train Epoch: 24  | Loss 1.920727
Train Epoch: 25  | Loss 1.539402
Train Epoch: 26  | Loss 1.538016
Train Epoch: 27  | Loss 1.333050
Train Epoch: 28  | Loss 1.203230
Train Epoch: 29  | Loss 1.070964
Train Epoch: 30  | Loss 1.083674


In [50]:
test_binary(model, test_input, test_target, 100)

Model Name : net2c

Test set:  Loss: 5.0983, Accuracy: 85%

